In [79]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [86]:
names = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_names.csv")

In [2]:
income = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_income.csv")

In [3]:
balance = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_balance-sheet.csv")

In [4]:
cashflow = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_cash-flow-statement.csv")

In [5]:
extra = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_extra.csv")

In [6]:
dates = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_dates.csv")

In [7]:
prices = pd.read_csv("C:/Users/Pavilion/GitHub/sp500/sp500_prices.csv")

In [8]:
financials = pd.concat([balance, income, cashflow], axis=0)
financials.columns = ['Ticker','Variable','Date','Value']
financials = financials[['Ticker','Date','Variable','Value']]
financials.head(1)

,Ticker,Date,Variable,Value
0,MMM,2024-09-30,Cash On Hand,"$7,295"


In [9]:
prices_cut = prices[['Company','Date','Close', 'Volume']]
prices_melt = prices_cut.melt(id_vars=['Company','Date'])
prices_melt['Date'] = prices_melt['Date'].str[:10]
prices_melt.columns = ['Ticker', 'Date', 'Variable', 'Value']
prices_melt.head(1)

,Ticker,Date,Variable,Value
0,MMM,1962-01-02,Close,0.565168


In [10]:
extra_proc = extra.drop(columns=['Unnamed: 0'])[['COMPANY','Date','Variable','Amount']]
extra_proc.columns = ['Ticker', 'Date', 'Variable', 'Value']
extra_proc.head(1)

,Ticker,Date,Variable,Value
0,MMM,2024-12-31,eps-earnings-per-share-diluted,$1.33


In [11]:
dates_proc = dates.drop(columns=['Unnamed: 0'])
dates_proc.head(1)

,Form Type,Filing Date,Ticker
0,10-Q,2024-10-22,MMM


In [12]:
sp500_10q = pd.concat([financials, extra_proc], axis=0)

In [13]:
sp500_10q["Date"] = pd.to_datetime(sp500_10q["Date"],format="%Y-%m-%d")
dates_proc["Filing Date"] = pd.to_datetime(dates_proc["Filing Date"],format="%Y-%m-%d")
sp500_10q = sp500_10q.sort_values("Date")
dates_proc = dates_proc.sort_values("Filing Date")
sp500_10q_fil = sp500_10q #pd.merge_asof(sp500_10q, dates_proc, left_on='Date', right_on='Filing Date', by='Ticker', direction='nearest')

In [14]:
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import ta

def compute_quarter_metrics(df, dates):
    df_prices = df.copy()
    records = []
    
    # Convert the "Date" column to datetime; drop rows that cannot be parsed.
    df_prices["Date"] = df_prices["Date"].astype('str').str[:10]
    
    # Loop through each unique ticker.
    for ticker in df_prices["Company"].unique():
        # Filter data for the given ticker
        df_ticker = df_prices[df_prices["Company"] == ticker].copy()
        
        # Ensure Date column is in datetime format
        df_ticker["Date"] = pd.to_datetime(df_ticker["Date"], format="%Y-%m-%d", errors="coerce")
        
        # Sort by Date to ensure correct order
        df_ticker = df_ticker.sort_values(by="Date", ascending=True)
        
        # Iterate over unique end_dates extracted from df_ticker
        for end_date in df_ticker.loc[df_ticker["Date"].dt.strftime("%m-%d").isin(dates), "Date"].dt.strftime("%Y-%m-%d").unique():
            
            # Skip if end_date is missing or empty
            if not end_date:
                continue
        
            # Convert end_date to datetime format safely
            end_date_dt = pd.to_datetime(end_date, format="%Y-%m-%d", errors="coerce")
            if pd.isnull(end_date_dt):  # Ensure valid date conversion
                continue
        
            # Ensure start_date is the first day of the month two months before end_date
            start_date = end_date_dt.replace(day=1) - relativedelta(months=2)
        
            # Filter using the ticker's Date column (ensure start_date is datetime)
            df_period = df_ticker[(df_ticker["Date"] >= start_date) & (df_ticker["Date"] <= end_date_dt)]
            
            # Skip if the filtered period is empty
            if df_period.empty:
                continue
            
            # For dividends, we replace a NaN result with 0.
            dividend_sum = df_period["Dividends"].sum()
            nonzero_count = (df_period["Dividends"] != 0).sum()
            mean_dividends = dividend_sum / nonzero_count if nonzero_count != 0 else 0
            
            # Calculate MACD and RSI
            macd = ta.trend.MACD(close=df_period["Close"], window_slow=26, window_fast=12, window_sign=9)
            rsi_14 = ta.momentum.RSIIndicator(close=df_period["Close"], window=14).rsi()
            
            metrics = {
                "Ticker": ticker,
                #"QuarterStart": start_date,
                "Date": end_date,
                #"NumDays": len(df_period),
                "ClosePrice": df_period.iloc[-1]["Close"],
                "MinPrice": df_period["Close"].min(),
                "MaxPrice": df_period["Close"].max(),
                "StdPrice": df_period["Close"].std(),
                "MeanPrice": df_period["Close"].mean(),
                "MedianPrice": df_period["Close"].median(),
                "MinVolume": df_period["Volume"].min(),
                "MaxVolume": df_period["Volume"].max(),
                "StdVolume": df_period["Volume"].std(),
                "MeanVolume": df_period["Volume"].mean(),
                "MedianVolume": df_period["Volume"].median(),
                "SumDividends": df_period["Dividends"].sum(),
                "MeanDividends": mean_dividends,
                "CountDividends": (df_period["Dividends"] != 0).sum(),
                "RSI_14": rsi_14.iloc[-1],
                "MACD": macd.macd().iloc[-1],
                "MACD_Signal": macd.macd_signal().iloc[-1],
                "MACD_Hist": macd.macd_diff().iloc[-1]
            }
            records.append(metrics) 
    return pd.DataFrame(records)

In [15]:
dates = sp500_10q_fil['Date'].astype('str').str[5:].unique()

In [16]:
sp500_10q_fil['Date'].astype('str').str[5:].unique()

array(['01-31', '02-28', '03-31', '04-30', '05-31', '06-30', '07-31',
       '08-31', '09-30', '10-31', '11-30', '12-31', '02-29'], dtype=object)

In [17]:
prices_agg = compute_quarter_metrics(prices, dates)

In [18]:
prices_agg_melt = prices_agg.melt(id_vars=['Ticker','Date'])

In [19]:
prices_agg_melt.columns = ['Ticker', 'Date', 'Variable', 'Value']

In [20]:
prices_agg_melt.head()

,Ticker,Date,Variable,Value
0,MMM,1962-01-31,ClosePrice,0.560919
1,MMM,1962-02-28,ClosePrice,0.569011
2,MMM,1962-03-31,ClosePrice,0.595650
3,MMM,1962-04-30,ClosePrice,0.541306
4,MMM,1962-05-31,ClosePrice,0.468848


In [21]:
sp500_merged = pd.concat([sp500_10q_fil, prices_agg_melt], axis=0)
sp500_merged.tail()

,Ticker,Date,Variable,Value
3749593,ZTS,2024-10-31,MACD_Hist,-0.816584
3749594,ZTS,2024-11-30,MACD_Hist,0.271507
3749595,ZTS,2024-12-31,MACD_Hist,-0.174002
3749596,ZTS,2025-01-31,MACD_Hist,0.582245
3749597,ZTS,2025-02-28,MACD_Hist,0.921411


In [22]:
def clean_numeric_column(series):
    # Convert all values to string and strip whitespace.
    s = series.astype(str).str.strip()
    # Remove dollar sign (the regex ensures that only the $ symbol is removed).
    s = s.str.replace(r'\$', '', regex=True)
    # Remove commas.
    s = s.str.replace(',', '', regex=True)
    # Replace values that are exactly "-" or empty with "0"
    s = s.replace({'-': '0', '': '0'})
    # Convert to numeric (this will preserve negative numbers)
    return pd.to_numeric(s, errors='coerce')

In [23]:
sp500_merged['Value'] = clean_numeric_column(sp500_merged['Value'])

In [24]:
sp500_merged['Date']= sp500_merged['Date'].astype('str').str[:10]

In [25]:
sp500_wide = sp500_merged[['Ticker', 'Date', 'Variable', 'Value']].pivot_table(index=["Ticker", "Date"], columns="Variable", values="Value")

In [26]:
sp500_wide.reset_index(inplace=True)

In [27]:
sp500_wide.head()

Variable,Ticker,Date,Basic EPS,Basic Shares Outstanding,Cash Flow From Financial Activities,Cash Flow From Investing Activities,Cash Flow From Operating Activities,Cash On Hand,Change In Accounts Payable,Change In Accounts Receivable,...,Total Current Assets,Total Current Liabilities,Total Depreciation And Amortization - Cash Flow,Total Liabilities,Total Liabilities And Share Holders Equity,Total Long Term Liabilities,Total Long-Term Assets,Total Non-Cash Items,Total Non-Operating Income/Expense,eps-earnings-per-share-diluted
0,A,1999-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2000-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,2000-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,2000-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
sp500_wide_clean = sp500_wide.dropna(subset=['Cash Flow From Operating Activities', 'ClosePrice'])

In [55]:
sp500_wide_clean_122024 = sp500_wide_clean.loc[sp500_wide_clean['Date']!='2024-12-31']

In [56]:
sp500_wide_clean_122024_fill = sp500_wide_clean_122024.fillna(0)

In [64]:
import pandas as pd
import numpy as np

def add_financial_ratios(data):
    """
    Adds common financial and price ratios to the given DataFrame in-place,
    using end-of-period values for metrics that are often averaged, and omitting
    ratios that require an Interest Expense field (which is not provided in the dataset).

    Required column names include (but are not limited to):
    - 'Total Current Assets'
    - 'Total Current Liabilities'
    - 'Cash On Hand'
    - 'Inventory'
    - 'Pre-Paid Expenses'
    - 'Cash Flow From Operating Activities'
    - 'Total Liabilities'
    - 'Share Holder Equity'
    - 'Long Term Debt'
    - 'EBIT'
    - 'Revenue'
    - 'Property, Plant, And Equipment'
    - 'Cost Of Goods Sold'
    - 'Receivables'
    - 'Gross Profit'
    - 'Operating Income'
    - 'Pre-Tax Income'
    - 'Net Income' or 'Net Income/Loss'
    - 'EBITDA'
    - 'Goodwill And Intangible Assets'
    - 'ClosePrice'
    - 'Shares Outstanding'
    - 'Common Stock Dividends Paid'
    - 'Net Change In Property, Plant, And Equipment'
    - 'Total Assets'

    If any needed column is missing or has zero in the denominator, the ratio is set to NaN.

    Returns:
        df: the same DataFrame with new ratio columns appended.
    """
    df = data.copy()
    
    def safe_div(num, denom):
        """Helper to handle division by zero or missing data."""
        if pd.isnull(num) or pd.isnull(denom) or denom == 0:
            return np.nan
        return num / denom

    # 1. Liquidity Ratios --------------------------------
    df['CurrentRatio'] = df.apply(
        lambda row: safe_div(
            row.get('Total Current Assets', np.nan),
            row.get('Total Current Liabilities', np.nan)
        ),
        axis=1
    )

    df['QuickRatio'] = df.apply(
        lambda row: safe_div(
            row.get('Total Current Assets', np.nan)
            - row.get('Inventory', 0)
            - row.get('Pre-Paid Expenses', 0),
            row.get('Total Current Liabilities', np.nan)
        ),
        axis=1
    )

    df['CashRatio'] = df.apply(
        lambda row: safe_div(
            row.get('Cash On Hand', np.nan),
            row.get('Total Current Liabilities', np.nan)
        ),
        axis=1
    )

    df['OperatingCashFlowRatio'] = df.apply(
        lambda row: safe_div(
            row.get('Cash Flow From Operating Activities', np.nan),
            row.get('Total Current Liabilities', np.nan)
        ),
        axis=1
    )

    # 2. Solvency (Leverage) Ratios ------------------------
    df['DebtToEquity'] = df.apply(
        lambda row: safe_div(
            row.get('Total Liabilities', np.nan),
            row.get('Share Holder Equity', np.nan)
        ),
        axis=1
    )

    df['DebtRatio'] = df.apply(
        lambda row: safe_div(
            row.get('Total Liabilities', np.nan),
            row.get('Total Assets', np.nan)
        ),
        axis=1
    )

    df['LongTermDebtToEquity'] = df.apply(
        lambda row: safe_div(
            row.get('Long Term Debt', np.nan),
            row.get('Share Holder Equity', np.nan)
        ),
        axis=1
    )

    df['LongTermDebtToCapital'] = df.apply(
        lambda row: safe_div(
            row.get('Long Term Debt', np.nan),
            row.get('Long Term Debt', 0) + row.get('Share Holder Equity', 0)
        ),
        axis=1
    )

    # 3. Efficiency (Activity) Ratios ---------------------
    df['AssetTurnover'] = df.apply(
        lambda row: safe_div(
            row.get('Revenue', np.nan),
            row.get('Total Assets', np.nan)
        ),
        axis=1
    )

    df['FixedAssetTurnover'] = df.apply(
        lambda row: safe_div(
            row.get('Revenue', np.nan),
            row.get('Property, Plant, And Equipment', np.nan)
        ),
        axis=1
    )

    # Inventory Turnover using end-of-period Inventory
    df['InventoryTurnover'] = df.apply(
        lambda row: safe_div(
            row.get('Cost Of Goods Sold', np.nan),
            row.get('Inventory', np.nan)
        ),
        axis=1
    )

    df['DaysInventoryOutstanding'] = 365 / df['InventoryTurnover']

    df['ReceivablesTurnover'] = df.apply(
        lambda row: safe_div(
            row.get('Revenue', np.nan),
            row.get('Receivables', np.nan)
        ),
        axis=1
    )

    df['DaysSalesOutstanding'] = 365 / df['ReceivablesTurnover']

    # 4. Profitability Ratios -----------------------------
    df['GrossMargin'] = df.apply(
        lambda row: safe_div(
            row.get('Gross Profit', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['OperatingMargin'] = df.apply(
        lambda row: safe_div(
            row.get('Operating Income', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['PreTaxProfitMargin'] = df.apply(
        lambda row: safe_div(
            row.get('Pre-Tax Income', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['NetProfitMargin'] = df.apply(
        lambda row: safe_div(
            row.get('Net Income', row.get('Net Income/Loss', np.nan)),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['EBITMargin'] = df.apply(
        lambda row: safe_div(
            row.get('EBIT', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['EBITDAMargin'] = df.apply(
        lambda row: safe_div(
            row.get('EBITDA', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['ROA'] = df.apply(
        lambda row: safe_div(
            row.get('Net Income', row.get('Net Income/Loss', np.nan)),
            row.get('Total Assets', np.nan)
        ),
        axis=1
    )

    df['ROE'] = df.apply(
        lambda row: safe_div(
            row.get('Net Income', row.get('Net Income/Loss', np.nan)),
            row.get('Share Holder Equity', np.nan)
        ),
        axis=1
    )

    df['ReturnOnTangibleEquity'] = df.apply(
        lambda row: safe_div(
            row.get('Net Income', row.get('Net Income/Loss', np.nan)),
            (
                row.get('Share Holder Equity', np.nan)
                - row.get('Goodwill And Intangible Assets', 0)
            )
        ),
        axis=1
    )

    df['ROCE'] = df.apply(
        lambda row: safe_div(
            row.get('EBIT', np.nan),
            (
                row.get('Total Assets', 0)
                - row.get('Total Current Liabilities', 0)
            )
        ),
        axis=1
    )

    df['ROI'] = df.apply(
        lambda row: safe_div(
            row.get('Net Income', row.get('Net Income/Loss', np.nan)),
            row.get('Share Holder Equity', 0) + row.get('Long Term Debt', 0)
        ),
        axis=1
    )

    # 5. Market Value Ratios ------------------------------
    df['MarketCap'] = df.apply(
        lambda row: safe_div(
            row.get('ClosePrice', np.nan) * row.get('Shares Outstanding', np.nan),
            1
        ),
        axis=1
    )

    def get_eps(row):
        if 'eps-earnings-per-share-diluted' in row and pd.notnull(row['eps-earnings-per-share-diluted']):
            return row['eps-earnings-per-share-diluted']
        return row.get('EPS - Earnings Per Share', np.nan)

    df['PE'] = df.apply(
        lambda row: safe_div(
            row.get('ClosePrice', np.nan),
            get_eps(row)
        ),
        axis=1
    )

    df['PriceToBook'] = df.apply(
        lambda row: safe_div(
            row.get('MarketCap', np.nan),
            row.get('Share Holder Equity', np.nan)
        ),
        axis=1
    )

    df['PriceToSales'] = df.apply(
        lambda row: safe_div(
            row.get('MarketCap', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['DividendPerShare'] = df.apply(
        lambda row: safe_div(
            row.get('Common Stock Dividends Paid', np.nan),
            row.get('Shares Outstanding', np.nan)
        ),
        axis=1
    )

    df['DividendYield'] = df.apply(
        lambda row: safe_div(
            row.get('DividendPerShare', np.nan),
            row.get('ClosePrice', np.nan)
        ),
        axis=1
    )

    df['DividendPayoutRatio'] = df.apply(
        lambda row: safe_div(
            row.get('DividendPerShare', np.nan),
            get_eps(row)
        ),
        axis=1
    )

    df['BookValuePerShare'] = df.apply(
        lambda row: safe_div(
            row.get('Share Holder Equity', np.nan),
            row.get('Shares Outstanding', np.nan)
        ),
        axis=1
    )

    # 6. Cash Flow–Specific Ratios ------------------------
    df['OperatingCashFlowToNetIncome'] = df.apply(
        lambda row: safe_div(
            row.get('Cash Flow From Operating Activities', np.nan),
            row.get('Net Income', row.get('Net Income/Loss', np.nan))
        ),
        axis=1
    )

    df['CashFlowMargin'] = df.apply(
        lambda row: safe_div(
            row.get('Cash Flow From Operating Activities', np.nan),
            row.get('Revenue', np.nan)
        ),
        axis=1
    )

    df['FreeCashFlow'] = df.apply(
        lambda row: (
            row.get('Cash Flow From Operating Activities', np.nan)
            - row.get('Net Change In Property, Plant, And Equipment', np.nan)
            if (
                not pd.isnull(row.get('Cash Flow From Operating Activities', np.nan))
                and not pd.isnull(row.get('Net Change In Property, Plant, And Equipment', np.nan))
            )
            else np.nan
        ),
        axis=1
    )

    df['FreeCashFlowYield'] = df.apply(
        lambda row: safe_div(
            row.get('FreeCashFlow', np.nan),
            row.get('MarketCap', np.nan)
        ),
        axis=1
    )

    df['OperatingCashFlowPerShare'] = df.apply(
        lambda row: safe_div(
            row.get('Cash Flow From Operating Activities', np.nan),
            row.get('Shares Outstanding', np.nan)
        ),
        axis=1
    )

    df['FreeCashFlowPerShare'] = df.apply(
        lambda row: safe_div(
            row.get('FreeCashFlow', np.nan),
            row.get('Shares Outstanding', np.nan)
        ),
        axis=1
    )

    return df

In [65]:
sp500_df = add_financial_ratios(sp500_wide_clean_122024_fill)

In [82]:
sp500_df.fillna(0, inplace = True)

In [91]:
sp500_df_names = sp500_df.merge(names[['Symbol', 'GICS Sector', 'GICS Sub-Industry', 'Founded']], how='left', left_on='Ticker', right_on='Symbol')

In [99]:
import pandas as pd
import numpy as np

def calc_quarterly_pct_diff(df, ticker_col='ticker', date_col='date', lags=[1, 4]):
    """
    Calculate the percentage change for each numeric column by ticker between periods 
    for given lags. For each numeric column, it computes the percent change for each lag 
    and appends new columns named "<col>_pct_diff_<lag>".
    
    The first row for each ticker is NaN for each lag since there's no previous period.

    Args:
        df (pd.DataFrame): The input DataFrame.
        ticker_col (str): Column name for ticker identifiers.
        date_col (str): Column name for date/time variable.
        lags (list): List of lag values (in periods) for which to compute the percent change.

    Returns:
        pd.DataFrame: A copy of 'df' with new percentage change columns appended.
    """
    # Sort by ticker/date to ensure correct chronological order
    df = df.sort_values(by=[ticker_col, date_col]).copy()

    # Identify numeric columns (exclude ticker/date from numeric, if present)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference([ticker_col, date_col])

    # Build a dict of new columns; each key corresponds to a new Series
    new_cols = {}

    # For each numeric column, compute percent change for each specified lag
    for col in numeric_cols:
        for lag in lags:
            new_col_name = f"{col}_pct_diff_{lag}"
            new_cols[new_col_name] = df.groupby(ticker_col)[col].pct_change(periods=lag)

    # Convert the dict to a DataFrame, aligning on the original index, then concat once
    df_new = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, df_new], axis=1)

    return df

In [100]:
sp500_diff = calc_quarterly_pct_diff(sp500_df_names, ticker_col='Ticker', date_col='Date')

In [102]:
sp500_diff.dropna(subset=['ClosePrice_pct_diff_1', 'ClosePrice_pct_diff_4'], inplace = True)

In [103]:
sp500_diff.fillna(0, inplace = True)

In [113]:
sp500_diff.drop(columns=['Symbol'], axis=1, inplace = True)

In [111]:
sp500_diff['Founded'] = sp500_diff['Founded'].str[0:4].astype('int')

In [114]:
sp500_diff.head()

,Ticker,Date,Basic EPS,Basic Shares Outstanding,Cash Flow From Financial Activities,Cash Flow From Investing Activities,Cash Flow From Operating Activities,Cash On Hand,Change In Accounts Payable,Change In Accounts Receivable,Change In Assets/Liabilities,Change In Inventories,ClosePrice,Common Stock Dividends Paid,Common Stock Net,Comprehensive Income,Cost Of Goods Sold,CountDividends,Debt Issuance/Retirement Net - Total,EBIT,EBITDA,EPS - Earnings Per Share,Financial Activities - Other,Goodwill And Intangible Assets,Gross Profit,Income After Taxes,Income From Continuous Operations,Income From Discontinued Operations,Income Taxes,Inventory,Investing Activities - Other,Long Term Debt,Long-Term Investments,MACD,MACD_Hist,MACD_Signal,MaxPrice,MaxVolume,MeanDividends,MeanPrice,MeanVolume,MedianPrice,MedianVolume,MinPrice,MinVolume,Net Acquisitions/Divestitures,Net Cash Flow,Net Change In Intangible Assets,Net Change In Investments - Total,Net Change In Long-Term Investments,"Net Change In Property, Plant, And Equipment",Net Change In Short-term Investments,Net Common Equity Issued/Repurchased,Net Current Debt,Net Income,Net Income/Loss,Net Long-Term Debt,Net Total Equity Issued/Repurchased,Operating Expenses,Operating Income,Other Current Assets,Other Income,Other Long-Term Assets,Other Non-Cash Items,Other Non-Current Liabilities,Other Operating Income Or Expenses,Other Share Holders Equity,Pre-Paid Expenses,Pre-Tax Income,"Property, Plant, And Equipment",RSI_14,Receivables,Research And Development Expenses,Retained Earnings (Accumulated Deficit),Revenue,SG&A Expenses,Share Holder Equity,Shares Outstanding,StdPrice,StdVolume,Stock-Based Compensation,SumDividends,Total Assets,Total Change In Assets/Liabilities,Total Common And Preferred Stock Dividends Paid,Total Current Assets,Total Current Liabilities,Total Depreciation And Amortization - Cash Flow,Total Liabilities,Total Liabilities And Share Holders Equity,Total Long Term Liabilities,Total Long-Term Assets,Total Non-Cash Items,Total Non-Operating Income/Expense,eps-earnings-per-share-diluted,CurrentRatio,QuickRatio,CashRatio,OperatingCashFlowRatio,DebtToEquity,DebtRatio,LongTermDebtToEquity,LongTermDebtToCapital,AssetTurnover,FixedAssetTurnover,InventoryTurnover,DaysInventoryOutstanding,ReceivablesTurnover,DaysSalesOutstanding,GrossMargin,OperatingMargin,PreTaxProfitMargin,NetProfitMargin,EBITMargin,EBITDAMargin,ROA,ROE,ReturnOnTangibleEquity,ROCE,ROI,MarketCap,PE,PriceToBook,PriceToSales,DividendPerShare,DividendYield,DividendPayoutRatio,BookValuePerShare,OperatingCashFlowToNetIncome,CashFlowMargin,FreeCashFlow,FreeCashFlowYield,OperatingCashFlowPerShare,FreeCashFlowPerShare,GICS Sector,GICS Sub-Industry,Founded,AssetTurnover_pct_diff_1,AssetTurnover_pct_diff_4,Basic EPS_pct_diff_1,Basic EPS_pct_diff_4,Basic Shares Outstanding_pct_diff_1,Basic Shares Outstanding_pct_diff_4,BookValuePerShare_pct_diff_1,BookValuePerShare_pct_diff_4,Cash Flow From Financial Activities_pct_diff_1,Cash Flow From Financial Activities_pct_diff_4,Cash Flow From Investing Activities_pct_diff_1,Cash Flow From Investing Activities_pct_diff_4,Cash Flow From Operating Activities_pct_diff_1,Cash Flow From Operating Activities_pct_diff_4,Cash On Hand_pct_diff_1,Cash On Hand_pct_diff_4,CashFlowMargin_pct_diff_1,CashFlowMargin_pct_diff_4,CashRatio_pct_diff_1,CashRatio_pct_diff_4,Change In Accounts Payable_pct_diff_1,Change In Accounts Payable_pct_diff_4,Change In Accounts Receivable_pct_diff_1,Change In Accounts Receivable_pct_diff_4,Change In Assets/Liabilities_pct_diff_1,Change In Assets/Liabilities_pct_diff_4,Change In Inventories_pct_diff_1,Change In Inventories_pct_diff_4,ClosePrice_pct_diff_1,ClosePrice_pct_diff_4,Common Stock Dividends Paid_pct_diff_1,Common Stock Dividends Paid_pct_diff_4,Common Stock Net_pct_diff_1,Common Stock Net_pct_diff_4,Comprehensive Income_pct_diff_1,Comprehensive Income_pct_diff_4,Cost Of Goods Sold_pct_diff_1,Cost Of Goods Sold_pct_diff_4,CountDividends_pct_diff_1,CountDivide

In [115]:
sp500_diff.to_csv('sp500_diff.csv')